<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B8/13/t2s_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata

auth_speech = userdata.get("AUTH_SPEECH")
auth_giga = userdata.get("SBER_AUTH")

In [2]:
!pip install -q gradio  ffmpeg-python
!pip install -q -U langchain_gigachat
import requests
import uuid

import wave
import numpy as np
import scipy.io
import gradio as gr
from IPython.display import Audio as IPythonAudio


import langchain
import langchain_gigachat

langchain.verbose = False
langchain.debug = False
langchain.llm_cache = False

from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, AIMessage

In [3]:
def get_token(auth_token, scope='SALUTE_SPEECH_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «SALUTE_SPEECH_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return None


In [4]:
response = get_token(auth_speech)
if response != None:
    salute_token = response.json()['access_token']

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Начать запись");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Остановить запись";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Идет сохранение"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


In [6]:
try:
  audio, sr = get_audio()
except:
  print('ОШИБКА ЗАПИСИ')

try:
  scipy.io.wavfile.write('recording.wav', sr, audio)
except:
  print('ОШИБКА сохранения')

sampling_rate, audio_data = scipy.io.wavfile.read('recording.wav')
sampling_rate

IPythonAudio(audio_data,rate=sampling_rate)

In [7]:
# Речь в текст
def stt(file_path):
    # URL для распознавания речи
    url = "https://smartspeech.sber.ru/rest/v1/speech:recognize"

    # Заголовки запроса
    headers = {
        "Authorization": f"Bearer {salute_token}",
        "Content-Type": "audio/x-pcm;bit=16;rate=16000"
    }

    # Открытие аудио файла в бинарном режиме
    with open(file_path, "rb") as audio_file:
        audio_data = audio_file.read()

    # Отправка POST запроса
    response = requests.post(url, headers=headers, data=audio_data, verify=False)

    # Обработка ответа
    if response.status_code == 200:
        result = response.json()
        print("Весь ответ API:", result)
        return result["result"][0]
    else:
        print("Ошибка:", response.status_code, response.text)
        return response.text

# Текст в речь
def synthesize_speech(text, token, file2save = 'output.wav', format='wav16', voice='Bys_24000'):
    """Генерация аудио из текста."""
    url = "https://smartspeech.sber.ru/rest/v1/text:synthesize"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/text"
    }
    params = {
        "format": format,
        "voice": voice
    }
    response = requests.post(url, headers=headers, params=params, data=text.encode(), verify=False)

    if response.status_code == 200:
        # Сохранение синтезированного аудио в файл
        with open(file2save, 'wb') as f:
            f.write(response.content)
        print("Аудио успешно синтезировано и сохранено как 'output.wav'")
        return response.content

# Класс, который определяет общую архитектуру аудио-чата с LLMкой.
class AudioChat:
    def __init__(self, auth_giga, salute_token):
        self.auth_giga = auth_giga
        self.salute_token = salute_token
        self.history = []
        self.llm = GigaChat(credentials=auth_giga, verify_ssl_certs=False)

    def conversation_predict(self, message):
        """Получаем ответ от LLM"""
        self.history.append(HumanMessage(content=message))
        gpt_response = self.llm.invoke(self.history)
        self.history.append(AIMessage(gpt_response.content))
        return gpt_response.content

    def process_audio2text(self, audio_input):
        """Конвертируем аудио в текст."""
        if audio_input is None:
            return ""
        sr, audio = audio_input
        scipy.io.wavfile.write("input.wav", sr, audio)
        rec_speech = stt("input.wav")
        return rec_speech

    def process_audio_to_text_and_speech(self, audio_input):
        """Общий метод для речь -> текст -> LLM -> текст -> речь"""
        prompt = self.process_audio2text(audio_input)
        print(prompt)
        answer = self.conversation_predict(prompt)
        print(answer)
        speech_answer = synthesize_speech(answer, self.salute_token)
        return (prompt, answer, speech_answer)

In [8]:
voice_chat = AudioChat(auth_giga, salute_token)

In [9]:
import gradio as gr
import numpy as np


# Универсальный метод для запроса LLMке (с использованием класса AudioChat)
def predict(audio, chat_history):
    if audio is None:
        return None, "", None, chat_history

    user_text, bot_text, bot_audio = voice_chat.process_audio_to_text_and_speech(audio)

    # Добавляем сообщения в формате Gradio 4+ (словари с role/content)
    chat_history.append({"role": "user", "content": user_text})
    chat_history.append({"role": "assistant", "content": bot_text})  # аудио — отдельно

    print(chat_history)

    return None, bot_text, bot_audio, chat_history

def clear_history():
    return None, "", None, []

with gr.Blocks(title="Голосовой чат с историей") as demo:
    gr.Markdown("## 🎙️ Голосовой чат-бот (Gradio ≥4.0)")

    with gr.Row():
        with gr.Column(scale=1):
            audio_input = gr.Audio(sources=["microphone"], type="numpy", label="Ваш голос")
            text_output = gr.Textbox(label="Текст ответа", interactive=False)
            audio_output = gr.Audio(label="Голосовой ответ", interactive=False)

            with gr.Row():
                btn_send = gr.Button("🎤 Отправить")
                btn_clear = gr.Button("🧹 Очистить")

        with gr.Column(scale=2):
            chatbot = gr.Chatbot(
                label="История диалога",
                type="messages"  # ← это включает новый формат
            )

    chat_state = gr.State([])  # хранит список сообщений в формате {"role": ..., "content": ...}

    btn_send.click(
        fn=predict,
        inputs=[audio_input, chat_state],
        outputs=[audio_input, text_output, audio_output, chat_state]
    ).then(
        fn=lambda history: history,
        inputs=chat_state,
        outputs=chatbot
    )

    btn_clear.click(
        fn=clear_history,
        inputs=[],
        outputs=[audio_input, text_output, audio_output, chat_state]
    ).then(
        fn=lambda: [],
        inputs=[],
        outputs=chatbot
    )

demo.launch(debug=True, share=True)

/tmp/ipython-input-2199950726.py:39: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7061649e453fe5a2af.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Весь ответ API: {'result': ['Привет.'], 'emotions': [{'negative': 0.011334172, 'neutral': 0.8795402, 'positive': 0.10912565}], 'person_identity': {'age': 'age_none', 'gender': 'gender_none', 'age_score': 0, 'gender_score': 0}, 'status': 200}
Привет.
Привет. Как настроение?


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'smartspeech.sber.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Аудио успешно синтезировано и сохранено как 'output.wav'
[{'role': 'user', 'content': 'Привет.'}, {'role': 'assistant', 'content': 'Привет. Как настроение?'}]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7061649e453fe5a2af.gradio.live
